In [1]:
from pathlib import Path
import csv
import pandas as pd
import json
from typing import NamedTuple, Dict, List, Tuple
from pprint import pprint
from operator import itemgetter
import re

DATASET = Path('/data/NER/VectorX/assignments_Разметка+ORG+2017-08-31_01-06-2018.tsv')
NE5Format = Path('/data/NER/VectorX/NE5_test')

san_dquoter = re.compile(r"(«|»|“|”|``|'|„|“)")

In [2]:
data = pd.read_csv(str(DATASET), delimiter='\t')
# with DATASET.open() as f:
#     r = csv.reader(f, delimiter='\t')
#     header = next(r)
#     for row in r:
#         pass

In [3]:
class EntityCandidate():
    start: int
    end: int # included
    options: Dict[Tuple[int], List[str]]
    
    def __init__(self, worker_id, start_index, end_index):
        self.start = start_index
        self.end = end_index
        self.options = {}
        self.add_option(worker_id, self.start, self.end)
        
    def is_same_entity(self, start_index, end_index):
        return (self.start <= start_index <= self.end) or \
               (self.start <= end_index <=self.end)
          
    def add_option(self, worker_id, start_index, end_index):
        """mutable function"""
        self.start = min(self.start, start_index)
        self.end = max(self.end, end_index)
        span = start_index, end_index
        if span not in self.options:
            self.options[span] = []
        self.options[span].append(worker_id)

In [45]:
re.match(r'^" *', '" word "').span()

(0, 2)

In [48]:
re.search(r' *"$', ' word ').span()

AttributeError: 'NoneType' object has no attribute 'span'

In [49]:

def get_spans(orig, words, tags):
    idx = 0
    last_word_end = -1
    span_start = -1
    span_end = -1
    spans = []
    for w, t in zip(words, tags):    
        while orig[idx] == ' ':
            idx += 1

        assert orig[idx:].startswith(w), [orig[idx:idx+10], w]

        if not w.strip():
            idx += len(w)
            continue

        if span_start != -1 and not t.startswith('I'):
            ent_candidate = orig[span_start:last_word_end]
            beg = re.match(r'^" *', ent_candidate)
            end = re.search(r' *"$', ent_candidate)
            
            if beg and end:
                print('!!!!!', ent_candidate)
                left_offset = beg.span()[1]
                ent_candidate = ent_candidate[left_offset:]
                span_start += left_offset
 
                right_offset = end.span()[1] - end.span()[0]
                ent_candidate = ent_candidate[:-right_offset]
                last_word_end -= right_offset
                
            spans.append([span_start, last_word_end, ent_candidate])
            span_start = -1    

        if t.startswith('B'):
            span_start = idx

        idx += len(w)
        last_word_end = idx
    return spans

In [50]:
for inp, d in data.groupby('INPUT:input'):
    orig = d['INPUT:orig'].iloc[0]
    guid = d['INPUT:guid'].iloc[0]
    words = inp.split(' ')
    tags = ['O' for _ in words]
    print(' '.join(words))

    entity_candidates = []
    for _, row in d.iterrows():
        idx = json.loads(row['OUTPUT:output'])
        wid = row['ASSIGNMENT:worker_id']
        
        for start, end in idx:
            for ec in entity_candidates:
                if ec.is_same_entity(start, end):
                    ec.add_option(wid, start, end)
                    break
            else:
                entity_candidates.append(EntityCandidate(wid, start, end))
                
    print()
    
    workers_count = max(1, len(d))
    for e in sorted(entity_candidates, key=lambda ec: ec.start):
        entity_founders_count = sum(len(v) for v in e.options.values())
#         print()
#         print(f'{entity_founders_count/workers_count:.2} ({entity_founders_count} / {workers_count})')
        if entity_founders_count >= 4:
            s, e = max(e.options.items(), key=lambda x: len(x[1]))[0]
            tags[s] = 'B-ORG'
            for i in range(s+1, e+1):
                tags[i] = 'I-ORG'
#             print(' '.join(words[s: e+1]), span, f'{score:.2}')
#         for span, workers in e.options.items():
#             score = len(workers) / workers_count
#             s, e = span
#             print(' '.join(words[s: e+1]), span, f'{score:.2} ({len(workers)}/{workers_count})')

    nice_orig = san_dquoter.sub('"', orig)
    spans = get_spans(nice_orig, words, tags)
    print(spans)
    with (NE5Format / f'{guid}.txt').open('w') as f:
        f.write(nice_orig)

    with (NE5Format / f'{guid}.ann').open('w') as f:
        w = csv.writer(f, delimiter='\t', quotechar='|')
        for i, span in enumerate(sorted(spans, key=itemgetter(0))):
            w.writerow([f'T{i+1}', f'ORG {span[0]} {span[1]}', orig[span[0]:span[1]]])


 
 
 
 
 Камеры видеонаблюдения помогли найти квартирного воришку . Мужчина совершил 45 краж в Архангельске и Северодвинске . При этом , некоторые пострадавшие узнали о краже только от сотрудников полиции . Злоумышленник ходил по подъездам и проверял - в каких квартирах открыты двери . После этого , снимал верхнюю одежду , заходил в прихожую , брал ценности и сумочки . Вычислить подставного соседа смог один из жителей многоквартирного дома . С помощью камер в подъезде , он увидел подозрительного мужчину и вызвал полицию . 
 Ольга Яшунова , юрист правового отдела УМВД России по Архангельской области : 
 - Видеонаблюдение является большим помощником в расследовании преступлений . Для того , чтобы установить видеонаблюдение во дворе жилого дома , либо в помещениях жилых домов , необходимо собрать собрание собственников , на которых установить количество установленных камер и места их размещения . 
 Известно , что мужчина уже не раз был судим за кражи . Сейчас подозреваемый находится в СИ

Запрос о созыве внеочередного заседания думы подали девять представителей новой коалиции , в которую вошли три представителя " Нашей партии " и шесть депутатов Латгальской партии . По закону , в такой ситуации заседание должно быть созвано в течение двух недель , то есть до 7 сентября . 
 
 
 24 августа Даугавпилсская дума не поддержала инициативу Элксниньша о смещении с должности вице - мэра Рихарда Эйгимса . 
 
 
 Несмотря на это , Элксниньш надеется сохранить в Даугавпилсской думе свою коалицию , предложив всем депутатам , кроме Эйгимса и бывшего мэра Яниса Лачплесиса ( Латгальская партия ) объединиться для борьбы с коррупцией . 
 
 
 В Даугавпилсской думе всего 15 депутатов , шестеро представляют Латгальскую партию , пять - " Согласие " и четыре " Нашу партию " .

!!!!! "Нашей партии"
!!!!! "Согласие"
!!!!! "Нашу партию"
[[125, 137, 'Нашей партии'], [157, 175, 'Латгальской партии'], [297, 316, 'Даугавпилсская дума'], [457, 476, 'Даугавпилсской думе'], [565, 583, 'Латгальская партия

Чистая прибыль Крестьянское ( фермерское ) хозяйство БАШИЛОВА ВЛАДИМИРА ВАСИЛЬЕВИЧА за 2015 год составила 21 , 00 тыс . руб . по сравнению с 51 , 00 тыс . руб . годом ранее . 
 
 
 Продажи компании сократились на 38 , 89 % до 110 , 00 тыс . руб . с 180 , 00 тыс . руб . годом ранее . 
 
 
 Компания Крестьянское ( фермерское ) хозяйство БАШИЛОВА ВЛАДИМИРА ВАСИЛЬЕВИЧА за 2015 год получила прибыль от продаж в 24 , 00 тыс . руб . , что в 2 , 21 раза меньше показателя за аналогичный период прошлого года . 
 
 
 Основным видом деятельности Крестьянское ( фермерское ) хозяйство БАШИЛОВА ВЛАДИМИРА ВАСИЛЬЕВИЧА ( ИНН - 6445000630 ) является выращивание зерновых , технических и прочих сельскохозяйственных культур , не включенных в другие группировки .

[[15, 81, 'Крестьянское (фермерское) хозяйство БАШИЛОВА ВЛАДИМИРА ВАСИЛЬЕВИЧА'], [274, 340, 'Крестьянское (фермерское) хозяйство БАШИЛОВА ВЛАДИМИРА ВАСИЛЬЕВИЧА'], [504, 570, 'Крестьянское (фермерское) хозяйство БАШИЛОВА ВЛАДИМИРА ВАСИЛЬЕВИЧА']]
Чист

In [42]:
entities = set()
for _, row in d.iterrows():
    idx = json.loads(row['OUTPUT:output'])
    words = inp.split(' ')
    for start, end in idx:
        e = ' '.join(words[start: end+1])
        entities.add(e)
print(entities)

set()


In [ ]:
words

In [ ]:
orig

In [ ]:
spans

In [ ]:
orig